<a href="https://colab.research.google.com/github/ChengZhenLi1/MAT-421/blob/main/HW_for_module_C_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chapter 19: Root finding**

**Section 19.1: Root finding problem statement**

Definition of root: The values of an independent variable in a function that will produce a zero are called roots

For example, in function f(x)=x^2-9, f(x) is 0 when x=-3,3. Therefore, -3 and 3 are the roots of the function.

For other functions that are diffcult to compute the roots of it, we use fsolve function from scipy to compute the roots or compute a approximated numerical value for roots

In [ ]:
import numpy as np
from scipy import optimize

f = lambda x: np.cos(x) - x
r = optimize.fsolve(f, -2)
print("r =", r)

# Verify the solution is a root
result = f(r)
print("result=", result)

r = [0.73908513]
result= [0.]


From the example above, the value of f(x) is 0 when r is 0.73908513. Therefore, 0.73908513 is the root of the function f(x)=cos(x)-x

When a function does have roots, the fsolve will generate a value for root but substitute the that value will not generate a 0.


In [ ]:
f = lambda x: 1/x

r, infodict, ier, mesg = optimize.fsolve(f, -2, full_output=True)
print("r =", r)

result = f(r)
print("result=", result)

print(mesg)

r = [-3.52047359e+83]
result= [-2.84052692e-84]
The number of calls to function has reached maxfev = 400.


From the example, the roots don't exist in this function, but fsolve still generate a value for root even the result is not a 0 when substitute it to the function.

**Section 19.2: Tolerance**

Definition of tolerance: It's the level of error that is acceptable for an enginerring application. 


For computing roots, there are 2 possible choices for the measure the error. One is to take the absolute value of f(x) as the measurement of error. The smaller a absolute value of f(x) is, the more likely the value of x is a root.Another way to measure the error is to take the absolute value of （x_(i+1)-x_i） as the measurement of error. The value of it will be smaller and smaller as x is approaching to the solution, which is the root of the function.


Example of the first method: Let f(x)= x^2 + tol/2, the measurement of error be "e= |f(x)|" and let the tolerance be "tol". f(x) doesn't have roots since there is a constant term. but f(0)=tol/2 is a acceptable solution for a root finding program since it's half of the tolerance and smaller than the tolerance.

Example of the second method: Let f(x)=1/x, the measurement of error be "e=|x_(i+1)-x_i|", and let the tolerance be "tol".  f(x) doesn't have a root since the denominator can not be 0 in any case. If let x_(i+1)=tol/4 and let x_i=-tol/4 then e=(2tol)/4=tol/2, which is smaller than the tolerance. Therefore it's an acceptable solution for a computer program.

**Section 19.3: Bisection Method**

Intermediate value theorem: If f(x) is a continuous function between a and b, and the sign of f(a) doesn't equal to the sign of f(b). Then there exists a c such that a < c < b and f(c)=0.

Bisection method: it's a method to use the intermediate value theorem repeatedly to find the roots of an function

The process of finding the root by using biseciton method is to first compute the midpoint of a interval to see if it's the root. Suppose the midpoint is m, if f(m) is 0, then m is the root. If f(m) > 0, then it means the the midpoint is in the interval (a,m). If f(m) < 0, it means the root is in the interval (m,b). By repeating this process, it can narrow down the interval where the root is in, then find the root after performing this process one or more times.   

In [10]:
import numpy as np

def my_bisection(f, a, b, tol): 
    # approximates a root, R, of f bounded 
    # by a and b to within tolerance 
    # | f(m) | < tol with m the midpoint 
    # between a and b Recursive implementation
    
    # check if a and b bound a root
    if np.sign(f(a)) == np.sign(f(b)):
        raise Exception(
         "The scalars a and b do not bound a root")
        
    # get midpoint
    m = (a + b)/2
    
    if np.abs(f(m)) < tol:
        # stopping condition, report m as root
        return m
    elif np.sign(f(a)) == np.sign(f(m)):
        # case where m is an improvement on a. 
        # Make recursive call with a = m
        return my_bisection(f, m, b, tol)
    elif np.sign(f(b)) == np.sign(f(m)):
        # case where m is an improvement on b. 
        # Make recursive call with b = m
        return my_bisection(f, a, m, tol)
  


In [23]:
f = lambda x: x**2 - 2

#set up the tolerance to be 0.1 and the interval to be (0,2)
r1 = my_bisection(f, 0, 2, 0.1)
print("r1 =", r1)

#repeating the process 
r01 = my_bisection(f, 0, 2, 0.01)
print("r01 =", r01)

#repeating the process again
r02 = my_bisection(f, 0, 2, 0.01)
print("r02 =", r02)
#repeating the process again
r03 = my_bisection(f, 0, 2, 0.01)
print("r03 =", r03)
print("f(r1) =", f(r1))
print("f(r01) =", f(r01))
print("f(r02) =", f(r02))
print("f(r03)=", f(r03))

r1 = 1.4375
r01 = 1.4140625
r02 = 1.4140625
r03 = 1.4140625
f(r1) = 0.06640625
f(r01) = -0.00042724609375
f(r02) = -0.00042724609375
f(r03)= -0.00042724609375


Setting up an reasonable interval is important for using this method, if the root is not in the initial interval, then it's impossible to find the root even after repeating the process multiple times.

In [24]:
my_bisection(f, 2, 4, 0.01)

Exception: ignored

From the above example, the error occurs in that line is because the root is not in the open interval (2,4)

**Section 19.4 Nweton-Raphson Method**

Definition: it's a method to find the root of a function by a process. The process is first pick a guess point (x_i) for the value of root. Then by proforming the equation (x_i)=[x_(i-1)]-[f(x_(i-1))/f(x_(i-1))' ] to see if the error is less than the tolerance. If it's not. finding the improvement by plugging the last x point. Until the error is less than the tolerance.When the error is less than the tolerance, use that approximated value for the root of the function.

In [1]:
import numpy as np

f = lambda x: x**2 - 2
f_prime = lambda x: 2*x
newton_raphson = 1.4 - (f(1.4))/(f_prime(1.4))

print("newton_raphson =", newton_raphson)
print("sqrt(2) =", np.sqrt(2))

newton_raphson = 1.4142857142857144
sqrt(2) = 1.4142135623730951


From the example above, it shows by using newton-raphson method, the approximated value for the root of function f(x)=x^2-2 is very close to its expect value square root of 2

One of the limitation for newton-raphson method is that if the denominator of the term [f(x_(i-1))/f(x_(i-1))' ] is close to 0. Then newton-raphson method will generate a very large guess that is very far from the value of the root.

Suppose the function f(X)=x^3+3*x^2-2*x-5 and the initial guess.

Then by using newton-raphson method:

In [3]:
x0 = 0.29
x1 = x0-(x0**4+3*x0**2-3*x0-5)/(3*x0**2+6*x0-2)
print("x1 =", x1)

x1 = -728.362881818157


It willl generate a value of -728.362881818157 for the next guess point. if we plug the value that newton-raphson method generated, the output is around -384239861. Which is extremely far from 0.

**Section 19.5: root finding in python**

In python, we can find the root of a function simply by using the function f_solve

In [5]:
from scipy.optimize import fsolve
f = lambda x: x**2-4

fsolve(f, [-5, 80])


array([-2.,  2.])

the roots for the function f(x)=x^2-4 is -2 and 2. As the function f_solve generated.